In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from torchtext.legacy.data import Field, TabularDataset, BucketIterator,ReversibleField
import matplotlib.pyplot as plt
from ast import literal_eval
import remi_utils as utils
import twoencodertransformer as kk
import pickle
source_folder = "solo_generation_dataset_augmented_mag"
folder = "dynamic_mag_models/2enc_2nd"
destination_folder = folder + "/solo_generation_weights"
generated_outputs = folder +  "/generated_samples"
dissimilar_interpolation = folder + "/interpolation"
vocab = folder + "/vocab"

In [2]:
from pathlib import Path
Path(destination_folder).mkdir(parents=True, exist_ok=True)
Path(generated_outputs).mkdir(parents=True, exist_ok=True)
Path(dissimilar_interpolation).mkdir(parents=True, exist_ok=True)
Path(vocab).mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/main").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/piano").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/solo").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/piano_predict").mkdir(parents=True, exist_ok=True)
Path(dissimilar_interpolation+"/intro").mkdir(parents=True, exist_ok=True)
Path(dissimilar_interpolation+"/outro").mkdir(parents=True, exist_ok=True)
Path(dissimilar_interpolation+"/predict").mkdir(parents=True, exist_ok=True)

In [3]:
event2word, word2event = pickle.load(open('dictionary_augmented.pkl', 'rb'))

In [4]:
if torch.cuda.is_available():  
    dev = "cuda:1" 
else:  
    dev = "cpu" 
print(dev)
device = torch.device(dev)
print(device)

cuda:1
cuda:1


In [5]:
# Fields

main_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
piano_field = Field(tokenize=None, lower=True, include_lengths=True, batch_first=True, init_token="<sos>", eos_token="<eos>")
fields = [('main', main_field), ('piano', piano_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train_torchtext.csv', validation='val_torchtext.csv', test='test_torchtext.csv',
                                           format='CSV', fields=fields, skip_header=True)

# Iterators
BATCH_SIZE = 1
train_iter = BucketIterator(train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.main),
                            device=device, sort=False, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.main),
                            device=device, sort=False, sort_within_batch=True)
test_iter = BucketIterator(test, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.main),
                            device=device, sort=False, sort_within_batch=True)

# Vocabulary

main_field.build_vocab(train, min_freq=1)
piano_field.build_vocab(train, min_freq=1)

In [6]:
big = []
for ((main, main_len), (piano, piano_len)), _ in (train_iter):
    #print(intro.transpose(1,0).size(0))
    print(piano_len.cpu().item())

1318
3668
1303
1071
945
1307
1660
1638
1435
2699
687
1278
1314
2699
1610
396
1059
682
2193
1335
798
929
1720
2023
1633
1890
1033
1765
1216
1141
602
1174
1250
1020
2332
1236
787
787
1941
843
1019
1250
800
1178
1084
1743
1690
1547
1715
1111
554
3783
967
1470
899
1425
1237
1176
1178
798
1313
1539
2792
1725
1303
898
1020
1335
1720
1246
895
1303
1030
1546
1644
1329
3528
687
2639
1768
758
906
1558
1542
858
1509
1075
816
1324
1399
705
1830
1960
1633
2117
1855
636
1598
1483
709
1271
1026
682
1355
1504
1047
1167
1335
2307
1086
838
1061
906
1135
1091
1351
439
914
2699
1207
1313
1376
1183
1199
1968
878
1542
1122
1086
931
869
1178
1180
1207
2112
2193
2087
914
2395
1429
868
1478
1419
905
1000
1313
839
822
1598
1180
1770
739
1144
1725
1278
727
1353
1086
1116
839
1442
804
802
1304
928
1019
1459
1113
2088
1318
1941
3668
1644
1774
355
1536
1323
1968
1250
726
1334
1643
2474
1116
1447
1443
1001
1478
929
1394
1509
1419
1638
530
1823
1026
757
1799
945
1607
549
1216
993
2418
794
1320
1935
1365
1113
794
1473

1171
1643
1116
2094
1456
1047
1355
705
1046
914
1180
1662
1558
1061
2087
2151
836
1900
1569
1166
1502
816
323
1999
1548
1470
1086
1798
1355
3783
1439
1274
854
821
1071
1365
1314
1010
1246
1654
439
1545
883
664
858
439
355
556
1443
1558
986
1166
2456
1004
836
1059
942
1046
588
1335
1275
1876
1557
929
1643
1211
1099
1320
530
1798
1026
2892
793
1718
1456
1439
1856
800
964
1478
1141
1443
1763
1387
1790
1274
745
1662
1656
942
1504
530
1322
1252
794
1355
672
1515
1790
1113
355
1376
1456
1303
2879
355
789
1496
1010
1395
1304
1956
1309
1246
2090
1178
687
1927
1508
914
1271
1508
859
854
940
2090
1383
1559
1998
1543
2398
1231
588
1746
1447
1329
828
1174
541
2639
1676
757
589
1438
1144
1353
810
1374
1380
1511
1180
931
1098
1504
874
1108
1920
1937
2117
878
793
1731
1084
1746
1963
1644
1351
787
1374
1001
1725
1595
1284
1134
1502
1319
1798
1539
1502
1509
2193
1086
1673
2094
2032
853
942
1928
1141
1037
1676
556
804
1542
467
934
1187
905
1167
1147
1124
2395
1211
1135
1900
1638
1690
845
1111
1309
638
1

In [7]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.enabled=False

In [8]:
#https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/more_advanced/seq2seq_transformer/seq2seq_transformer.py
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out


In [9]:
src_vocab_size = len(main_field.vocab)
trg_vocab_size = len(piano_field.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 3000
forward_expansion = 4
src_pad_idx = 1 #english.vocab.stoi["<pad>"]

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
)
model = model.to(device)


In [10]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=1e-5) #non augmented 3e-4


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


def save_best_checkpoint(state, nth,filename="_checkpoint.pt"):
    print("=> Saving checkpoint")
#     torch.save(state, destination_folder + str(nth)+filename)
    torch.save(state, destination_folder + '/metrics.pt')

def save_final_checkpoint(state, nth,filename="_checkpoint.pt"):
    print("=> Saving checkpoint")
    torch.save(state, destination_folder + "/" + str(nth)+filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

The model has 12,969,244 trainable parameters


In [11]:
# stoi input str get int
# intro_field.vocab.stoi
# itos input into get token/str
# intro_field.vocab.itos[4]

In [12]:
PAD_IDX = 1

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
#criterion = nn.CrossEntropyLoss()

In [13]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    #for _, (src, _,trg,_) in enumerate(iterator):
    for ((main, main_len), (piano, piano_len)), _ in (iterator):
        if piano_len.cpu().item()>=3000:
            continue
        src, trg = main.transpose(1,0), piano.transpose(1,0)
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()
        output = model(src, trg[:-1, :])
        
#         print(output.size())
#         print(trg.size())
        
        output = output.view(-1, output.shape[-1])
        trg = trg[1:].reshape(-1)
        loss = criterion(output, trg)
#         print(torch.isfinite(trg).all().cpu().item())
#         print(torch.isfinite(output).all().cpu().item())
#         print(torch.isfinite(loss).all().cpu().item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.cpu().detach().item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        #for _, (src, _,trg,_) in enumerate(iterator):
        for ((main, main_len), (piano, piano_len)), _ in (iterator):
            if piano_len.cpu().item()>=3000:
                continue
            src, trg = main.transpose(1,0), piano.transpose(1,0)
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg[:-1, :]) #turn off teacher forcing

            output = output.view(-1, output.shape[-1])
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.cpu().detach().item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [14]:
def translate_sentence(model, sentence, german, english, device, max_length=1200):

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break
    # print(outputs)
    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence


In [15]:
df_intro = pd.read_csv(source_folder + '/val_torchtext.csv')
val_main = df_intro['main'].values
val_piano = df_intro['piano'].values
val_data=[]
for i in range(len(val_main)):
    temp_dict = {}
    temp_dict['main'] = val_main[i]
    temp_dict['piano'] = val_piano[i]
    val_data.append(temp_dict)
print(len(val_piano))

112


In [16]:
def check_mode_collapse(model):
    count = 0
    translations = []
    for i in range(3):
        main = val_main[i]
        piano = val_piano[i]
        #print(intro)
        list_main = [int(x) for x in main.split(' ')]
        list_piano = [int(x) for x in piano.split(' ')]
        translated_sentence = translate_sentence(model, main, main_field, piano_field, device, max_length=1200)
        
        translated_sentence = [int(x) for x in translated_sentence if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']
        print(translated_sentence)
        translations.append(translated_sentence)
        if i > 0:
            if translations[i-1] == translations[i]:
                count += 1
    return count


In [ ]:
N_EPOCHS = 2000
S_EPOCH = 0
CLIP = 1

train_loss_log = []
valid_loss_log = []
best_valid_loss = float('inf')
#torch.autograd.set_detect_anomaly(True)
#model = nn.DataParallel(model, device_ids=[0,1]).to(device)
for epoch in range(S_EPOCH, N_EPOCHS):
    
    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    
    train_loss_log.append(train_loss)
    valid_loss_log.append(valid_loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        checkpoint = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
        save_best_checkpoint(checkpoint,N_EPOCHS)
    if (epoch+1) % 20 == 0 or (epoch) % 20 == 0:
        save_final_checkpoint(checkpoint,epoch)
    if (epoch+1) % 25 ==0:
        if check_mode_collapse(model) > 1:
            print("model is mode collapsing")
save_final_checkpoint(checkpoint,N_EPOCHS)
test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 5m 33s
	Train Loss: 4.891 | Train PPL: 133.078
	 Val. Loss: 4.393 |  Val. PPL:  80.906
=> Saving checkpoint
=> Saving checkpoint
Epoch: 02 | Time: 5m 37s
	Train Loss: 4.150 | Train PPL:  63.456
	 Val. Loss: 3.786 |  Val. PPL:  44.088
=> Saving checkpoint
Epoch: 03 | Time: 5m 38s
	Train Loss: 3.531 | Train PPL:  34.148
	 Val. Loss: 3.172 |  Val. PPL:  23.848
=> Saving checkpoint
Epoch: 04 | Time: 5m 38s
	Train Loss: 3.108 | Train PPL:  22.366
	 Val. Loss: 2.908 |  Val. PPL:  18.324
=> Saving checkpoint
Epoch: 05 | Time: 5m 39s
	Train Loss: 2.925 | Train PPL:  18.627
	 Val. Loss: 2.768 |  Val. PPL:  15.929
=> Saving checkpoint
Epoch: 06 | Time: 5m 39s
	Train Loss: 2.818 | Train PPL:  16.744
	 Val. Loss: 2.669 |  Val. PPL:  14.420
=> Saving checkpoint
Epoch: 07 | Time: 5m 39s
	Train Loss: 2.734 | Train PPL:  15.387
	 Val. Loss: 2.585 |  Val. PPL:  13.261
=> Saving checkpoint
Epoch: 08 | Time: 5m 39s
	Train Loss: 2.659 | Train PPL:  14.281
	 Val. Loss: 2.531 |  Val. PPL: 

[0, 1, 2, 114, 1, 51, 111, 15, 1, 51, 111, 15, 1, 51, 33, 15, 4, 51, 33, 15, 8, 51, 111, 15, 8, 51, 111, 15, 8, 51, 111, 15, 13, 51, 33, 15, 13, 51, 33, 15, 13, 51, 33, 15, 13, 51, 33, 15, 17, 51, 33, 15, 17, 51, 33, 15, 17, 51, 33, 15, 17, 51, 33, 15, 17, 51, 6, 15, 0, 1, 51, 33, 15, 1, 51, 33, 15, 1, 51, 6, 15, 1, 51, 33, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 8, 51, 6, 15, 8, 51, 6, 15, 8, 51, 6, 15, 8, 51, 6, 15, 8, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 13, 51, 6, 15, 16, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 6, 15, 17, 51, 33, 15, 0, 1, 51, 6, 15, 0, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1, 51, 6, 15, 1

[0, 1, 2, 130, 67, 54, 128, 15, 67, 54, 128, 15, 23, 54, 100, 15, 23, 54, 128, 15, 72, 54, 128, 15, 72, 54, 128, 15, 91, 54, 128, 15, 91, 54, 128, 15, 70, 54, 128, 15, 70, 54, 128, 15, 70, 54, 128, 15, 78, 54, 128, 15, 78, 54, 128, 15, 90, 54, 128, 15, 74, 54, 128, 15, 74, 54, 128, 15, 74, 54, 128, 15, 74, 54, 128, 15, 74, 54, 128, 15, 23, 54, 128, 15, 72, 54, 128, 15, 72, 54, 128, 15, 91, 54, 128, 15, 91, 54, 128, 15, 13, 54, 128, 15, 70, 54, 128, 15, 70, 54, 128, 15, 17, 54, 128, 15, 78, 54, 128, 15, 74, 51, 128, 15, 74, 54, 128, 15, 91, 54, 128, 15, 91, 51, 128, 15, 78, 51, 128, 15, 78, 54, 128, 15, 74, 51, 128, 15, 17, 54, 128, 7, 74, 51, 128, 15, 74, 51, 128, 15, 74, 54, 128, 15, 23, 54, 128, 15, 23, 54, 128, 15, 72, 51, 128, 15, 72, 51, 128, 15, 72, 51, 128, 15, 91, 54, 128, 15, 70, 54, 128, 15, 70, 51, 128, 15, 91, 54, 128, 15, 70, 51, 128, 15, 70, 51, 128, 15, 78, 54, 128, 15, 78, 51, 128, 15, 74, 51, 128, 15, 74, 54, 128, 15, 74, 54, 128, 15, 74, 51, 128, 15, 74, 51, 128, 15, 

Epoch: 51 | Time: 5m 40s
	Train Loss: 2.077 | Train PPL:   7.981
	 Val. Loss: 2.523 |  Val. PPL:  12.465
Epoch: 52 | Time: 5m 40s
	Train Loss: 2.072 | Train PPL:   7.937
	 Val. Loss: 2.532 |  Val. PPL:  12.584
Epoch: 53 | Time: 5m 40s
	Train Loss: 2.066 | Train PPL:   7.892
	 Val. Loss: 2.553 |  Val. PPL:  12.844
Epoch: 54 | Time: 5m 39s
	Train Loss: 2.060 | Train PPL:   7.849
	 Val. Loss: 2.547 |  Val. PPL:  12.763
Epoch: 55 | Time: 5m 40s
	Train Loss: 2.055 | Train PPL:   7.807
	 Val. Loss: 2.556 |  Val. PPL:  12.885
Epoch: 56 | Time: 5m 40s
	Train Loss: 2.050 | Train PPL:   7.771
	 Val. Loss: 2.560 |  Val. PPL:  12.933
Epoch: 57 | Time: 5m 40s
	Train Loss: 2.045 | Train PPL:   7.728
	 Val. Loss: 2.576 |  Val. PPL:  13.139
Epoch: 58 | Time: 5m 40s
	Train Loss: 2.039 | Train PPL:   7.685
	 Val. Loss: 2.565 |  Val. PPL:  12.995
Epoch: 59 | Time: 5m 40s
	Train Loss: 2.034 | Train PPL:   7.644
	 Val. Loss: 2.580 |  Val. PPL:  13.198
Epoch: 60 | Time: 5m 40s
	Train Loss: 2.029 | Train PPL

[0, 1, 2, 69, 1, 51, 111, 52, 4, 51, 111, 52, 4, 51, 33, 52, 4, 51, 33, 52, 8, 51, 33, 52, 8, 51, 33, 52, 10, 51, 33, 52, 10, 51, 33, 52, 10, 51, 33, 52, 13, 51, 33, 52, 13, 51, 33, 52, 16, 51, 33, 52, 16, 51, 33, 52, 17, 51, 33, 52, 17, 51, 33, 52, 17, 51, 33, 52, 27, 51, 33, 52, 0, 1, 51, 33, 31, 1, 51, 33, 52, 4, 51, 33, 52, 4, 51, 33, 52, 8, 51, 33, 52, 8, 51, 33, 52, 8, 51, 33, 52, 10, 51, 33, 52, 13, 51, 33, 52, 13, 51, 33, 52, 13, 51, 33, 52, 16, 51, 33, 52, 16, 51, 33, 52, 78, 51, 33, 52, 17, 51, 33, 52, 17, 51, 33, 52, 27, 51, 33, 52, 0, 1, 51, 33, 52, 4, 51, 33, 52, 4, 51, 33, 52, 8, 51, 33, 52, 8, 51, 33, 52, 8, 51, 33, 52, 13, 51, 33, 52, 13, 51, 33, 52, 16, 51, 6, 31, 17, 51, 6, 52, 17, 51, 33, 52, 0, 1, 51, 33, 52, 1, 51, 33, 52, 4, 51, 6, 52, 8, 51, 33, 52, 8, 51, 33, 52, 8, 51, 6, 52, 8, 51, 33, 52, 13, 51, 33, 31, 13, 51, 33, 52, 13, 51, 33, 31, 16, 51, 6, 52, 17, 51, 33, 52, 17, 51, 33, 52, 17, 51, 33, 52, 17, 51, 33, 52, 0, 1, 51, 6, 52, 4, 51, 33, 52, 4, 51, 33, 31,

...

[0, 1, 2, 130, 67, 49, 128, 15, 67, 51, 100, 15, 23, 51, 128, 7, 23, 54, 185, 19, 72, 54, 128, 15, 72, 51, 100, 7, 91, 51, 55, 22, 72, 51, 178, 47, 91, 51, 100, 31, 70, 54, 128, 15, 78, 51, 100, 15, 90, 51, 30, 52, 90, 51, 132, 19, 74, 51, 128, 52, 74, 54, 100, 15, 0, 67, 51, 178, 59, 67, 35, 122, 19, 23, 51, 100, 19, 72, 53, 122, 7, 91, 53, 100, 7, 70, 54, 30, 19, 78, 51, 122, 7, 90, 54, 100, 52, 74, 51, 30, 52, 0, 67, 51, 218, 41, 23, 51, 111, 7, 72, 64, 30, 7, 91, 64, 111, 15, 70, 49, 180, 19, 78, 64, 128, 19, 78, 64, 111, 52, 90, 51, 55, 52, 74, 51, 48, 52, 74, 51, 111, 15, 0, 67, 53, 55, 15, 23, 54, 178, 66, 23, 54, 122, 19, 72, 64, 100, 19, 72, 49, 30, 7, 91, 54, 122, 7, 70, 53, 149, 19, 78, 54, 132, 19, 78, 54, 128, 19, 78, 35, 100, 52, 90, 54, 30, 52, 90, 51, 227, 19, 74, 51, 122, 52, 0, 67, 49, 178, 19, 23, 53, 122, 19, 72, 53, 55, 52, 72, 51, 129, 7, 91, 54, 122, 52, 91, 54, 55, 52, 70, 51, 129, 7, 78, 54, 122, 52, 78, 35, 122, 15, 90, 35, 55, 52, 74, 51, 185, 19, 74, 54, 128

Epoch: 1601 | Time: 5m 43s
	Train Loss: 0.131 | Train PPL:   1.140
	 Val. Loss: 7.237 |  Val. PPL: 1390.114
=> Saving checkpoint
Epoch: 1602 | Time: 5m 42s
	Train Loss: 0.131 | Train PPL:   1.139
	 Val. Loss: 7.303 |  Val. PPL: 1484.883
Epoch: 1603 | Time: 5m 42s
	Train Loss: 0.130 | Train PPL:   1.139
	 Val. Loss: 7.260 |  Val. PPL: 1422.152
Epoch: 1604 | Time: 5m 42s
	Train Loss: 0.131 | Train PPL:   1.139
	 Val. Loss: 7.279 |  Val. PPL: 1449.164
Epoch: 1605 | Time: 5m 42s
	Train Loss: 0.131 | Train PPL:   1.139
	 Val. Loss: 7.269 |  Val. PPL: 1435.346
Epoch: 1606 | Time: 5m 42s
	Train Loss: 0.130 | Train PPL:   1.139
	 Val. Loss: 7.274 |  Val. PPL: 1441.696
Epoch: 1607 | Time: 5m 42s
	Train Loss: 0.130 | Train PPL:   1.138
	 Val. Loss: 7.262 |  Val. PPL: 1425.721
Epoch: 1608 | Time: 5m 42s
	Train Loss: 0.130 | Train PPL:   1.138
	 Val. Loss: 7.260 |  Val. PPL: 1421.728
Epoch: 1609 | Time: 5m 42s
	Train Loss: 0.130 | Train PPL:   1.138
	 Val. Loss: 7.230 |  Val. PPL: 1380.497
Epoch: 

[0, 1, 2, 162, 1, 49, 55, 31, 1, 53, 33, 31, 1, 54, 50, 31, 4, 64, 57, 31, 4, 54, 50, 15, 8, 51, 55, 47, 8, 64, 6, 15, 8, 51, 33, 31, 8, 54, 50, 7, 72, 51, 55, 15, 10, 51, 128, 7, 91, 32, 129, 36, 13, 54, 33, 15, 13, 64, 50, 31, 70, 53, 55, 7, 16, 51, 33, 7, 16, 64, 50, 15, 78, 64, 55, 15, 27, 51, 50, 41, 27, 51, 33, 7, 74, 35, 50, 7, 0, 1, 49, 55, 15, 1, 64, 55, 15, 1, 54, 33, 7, 1, 64, 50, 7, 4, 51, 55, 15, 23, 54, 50, 7, 8, 54, 33, 15, 8, 54, 57, 15, 10, 54, 129, 36, 10, 35, 55, 52, 13, 64, 33, 7, 13, 53, 50, 31, 16, 51, 55, 7, 17, 54, 33, 7, 17, 54, 50, 31, 27, 51, 57, 15, 0, 1, 64, 129, 103, 1, 35, 33, 31, 1, 54, 50, 52, 1, 51, 57, 31, 4, 53, 33, 7, 8, 49, 55, 41, 8, 53, 50, 15, 10, 35, 57, 7, 10, 54, 33, 15, 13, 64, 55, 15, 13, 51, 180, 34, 13, 64, 55, 31, 16, 32, 33, 31, 16, 51, 128, 28, 16, 49, 62, 31, 17, 35, 55, 31, 17, 51, 48, 31, 27, 35, 6, 7, 0, 1, 54, 180, 125, 1, 45, 55, 41, 4, 53, 33, 7, 8, 49, 6, 52, 8, 53, 75, 31, 8, 49, 111, 19, 10, 71, 55, 19, 10, 35, 33, 19, 13, 54

[0, 1, 2, 130, 67, 49, 128, 15, 67, 51, 100, 15, 23, 51, 129, 15, 23, 49, 185, 47, 72, 54, 128, 52, 72, 51, 55, 52, 91, 51, 100, 22, 70, 49, 128, 15, 70, 51, 55, 15, 78, 54, 128, 22, 78, 54, 111, 15, 90, 32, 55, 7, 90, 64, 48, 7, 74, 51, 178, 59, 0, 67, 51, 122, 15, 67, 64, 100, 7, 23, 53, 149, 7, 8, 54, 100, 15, 72, 53, 30, 15, 72, 54, 50, 15, 91, 64, 30, 15, 70, 53, 122, 22, 70, 53, 100, 22, 78, 54, 30, 22, 90, 64, 48, 22, 74, 51, 178, 66, 0, 67, 54, 100, 15, 67, 51, 122, 7, 23, 49, 100, 52, 23, 51, 100, 15, 72, 51, 30, 15, 72, 51, 100, 15, 91, 54, 30, 15, 70, 53, 50, 15, 78, 54, 218, 66, 78, 53, 111, 7, 78, 54, 30, 19, 90, 54, 48, 41, 74, 51, 111, 7, 74, 49, 149, 52, 0, 67, 54, 128, 15, 23, 54, 132, 19, 23, 51, 100, 7, 23, 51, 48, 52, 72, 54, 100, 52, 91, 54, 30, 19, 91, 51, 48, 19, 70, 35, 100, 15, 78, 51, 129, 31, 78, 53, 178, 19, 78, 51, 122, 7, 90, 51, 100, 52, 74, 51, 30, 52, 0, 67, 51, 129, 19, 67, 51, 122, 15, 67, 49, 55, 15, 23, 53, 46, 19, 72, 51, 122, 15, 91, 54, 100, 7, 7

Epoch: 1652 | Time: 5m 41s
	Train Loss: 0.124 | Train PPL:   1.132
	 Val. Loss: 7.368 |  Val. PPL: 1585.054
Epoch: 1653 | Time: 5m 42s
	Train Loss: 0.124 | Train PPL:   1.132
	 Val. Loss: 7.348 |  Val. PPL: 1552.718
Epoch: 1654 | Time: 5m 42s
	Train Loss: 0.124 | Train PPL:   1.132
	 Val. Loss: 7.360 |  Val. PPL: 1572.330
Epoch: 1655 | Time: 5m 42s
	Train Loss: 0.124 | Train PPL:   1.132
	 Val. Loss: 7.336 |  Val. PPL: 1533.879
Epoch: 1656 | Time: 5m 43s
	Train Loss: 0.124 | Train PPL:   1.132
	 Val. Loss: 7.334 |  Val. PPL: 1531.370
Epoch: 1657 | Time: 5m 42s
	Train Loss: 0.123 | Train PPL:   1.131
	 Val. Loss: 7.359 |  Val. PPL: 1570.448
Epoch: 1658 | Time: 5m 42s
	Train Loss: 0.123 | Train PPL:   1.131
	 Val. Loss: 7.391 |  Val. PPL: 1621.943
Epoch: 1659 | Time: 5m 42s
	Train Loss: 0.124 | Train PPL:   1.131
	 Val. Loss: 7.368 |  Val. PPL: 1584.065
Epoch: 1660 | Time: 5m 42s
	Train Loss: 0.123 | Train PPL:   1.131
	 Val. Loss: 7.383 |  Val. PPL: 1609.021
=> Saving checkpoint
Epoch: 

In [18]:
# checkpoint = {'model_state_dict': model.state_dict(),
#                   'optimizer_state_dict': optimizer.state_dict(),
#                   'valid_loss': valid_loss}
# save_checkpoint(destination_folder + checkpoint,N_EPOCHS)

In [ ]:
output = open(folder + "/train_loss_log.pkl", 'wb')
pickle.dump(train_loss_log, output)
output.close()

output = open(folder + "/valid_loss_log.pkl", 'wb')
pickle.dump(valid_loss_log, output)
output.close()

In [19]:
best_model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)
optimizer = optim.Adam(best_model.parameters(), lr=0.001)

In [ ]:
state = torch.load(destination_folder + '/1000_checkpoint.pt', map_location=device)
load_checkpoint(state, model, optimizer)

In [ ]:
test_loss = evaluate(model, test_iter, criterion)
print(math.exp(test_loss))

In [ ]:
generated_outputs = folder +  "/generated_samples_1000epochs"
Path(generated_outputs+"/main").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/piano").mkdir(parents=True, exist_ok=True)
Path(generated_outputs+"/piano_predict").mkdir(parents=True, exist_ok=True)

In [ ]:

df_intro = pd.read_csv(source_folder + '/test_torchtext.csv')
test_main = df_intro['main'].values
test_piano = df_intro['piano'].values
test_data=[]
for i in range(len(val_main)):
    temp_dict = {}
    temp_dict['main'] = test_main[i]
    temp_dict['piano'] = test_piano[i]
    test_data.append(temp_dict)
print(len(test_piano))

In [ ]:
for i in range(0,len(test_piano)):
#     if len(test_intro) > 1200:
#         continue
    main = test_main[i]
    piano = test_piano[i]
    list_main = [int(x) for x in main.split(' ')]
    list_piano = [int(x) for x in piano.split(' ')]
    
    translated_sentence = translate_sentence(model, main, main_field, piano_field, device, max_length=3000)
        #print(translated_sentence)
    translated_sentence = [int(x) for x in translated_sentence if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']
    print(translated_sentence)
    utils.write_midi(list_main, word2event, generated_outputs + "/main/" + "/main" + str(i)  + ".mid")
    utils.write_midi(list_piano, word2event, generated_outputs  + "/piano/" + "/piano" + str(i)  + ".mid")
    utils.write_midi(translated_sentence, word2event, generated_outputs + "/piano_predict/" + "/piano_predict" + str(i)  + ".mid")
    print(i)
#     if i == 10:
#         break
        


In [ ]:
import mido
for i in range(11):
    piano = mido.MidiFile(generated_outputs + "/piano/" + '/piano' + str(i) + '.mid')
    main = mido.MidiFile(generated_outputs + "/main/" +'/main' + str(i) + '.mid')
    predict = mido.MidiFile(generated_outputs + "/piano_predict/" +'/piano_predict' + str(i) + '.mid')

    piano.tracks[1].name = "piano"
    main.tracks[1].name = "main"
    predict.tracks[1].name = "piano_predict"
    merged_mid = mido.MidiFile()
    merged_mid.ticks_per_beat = main.ticks_per_beat
    merged_mid.tracks = piano.tracks + main.tracks 
    merged_mid.save(generated_outputs + '/merged' + str(i) + '.mid')
    
    merged_mid = mido.MidiFile()
    merged_mid.ticks_per_beat = main.ticks_per_beat
    merged_mid.tracks = predict.tracks + main.tracks 
    merged_mid.save(generated_outputs + '/merged_predict' + str(i) + '.mid')

In [26]:
# dissimilar_interpolation
for i in range(0,len(test_intro)):
#     if len(test_intro) > 1200:
#         continue
    intro = test_intro[i]
    #solo = test_solo[i]
    if i + 3 < (len(test_intro)):
        outro = test_outro[i+3]
    else:
        outro = test_outro[i]
    #print(intro)
    #print(outro)
    list_intro = [int(x) for x in intro.split(' ')]
    #list_solo = [int(x) for x in solo.split(' ')]
    list_outro = [int(x) for x in outro.split(' ')]
    #print(list_sentence)
    translated_sentence = translate_sentence(model, intro, outro, intro_field, outro_field, solo_field, device, max_length=1200)
    #print(translated_sentence)
    translated_sentence = [int(x) for x in translated_sentence if x != '<pad>' and x != '<sos>' and x != '<eos>' and x != '<unk>']
    print(translated_sentence)
    utils.write_midi(list_intro, word2event, dissimilar_interpolation + "/intro/" + "/intro" + str(i)  + ".mid")
    #utils.write_midi(list_solo, word2event, generated_outputs  + "/solo/" + "/solo" + str(i)  + ".mid")
    utils.write_midi(list_outro, word2event, dissimilar_interpolation + "/outro/" + "/outro" + str(i)  + ".mid")
    utils.write_midi(translated_sentence, word2event, dissimilar_interpolation + "/predict/" + "/predict" + str(i)  + ".mid")
    print(i)
#     if i == 10:
#         break
        


NameError: name 'test_intro' is not defined

In [ ]:
import mido
for i in range(len(test_intro)):
    intro = mido.MidiFile(dissimilar_interpolation + "/intro/" + '/intro' + str(i) + '.mid')
    outro = mido.MidiFile(dissimilar_interpolation + "/outro/" +'/outro' + str(i) + '.mid')
    predict = mido.MidiFile(dissimilar_interpolation + "/predict/" +'/predict' + str(i) + '.mid')
    total_intro_time = 0
    total_solo_time = 0
    total_predict_time = 0
    for msg in intro.tracks[1]:
        if msg.type == "note_on":
            total_intro_time += msg.time
    for msg in predict.tracks[1]:
        if msg.type == "note_on":
            total_predict_time += msg.time
            
    original_outro_time = 0 + outro.tracks[1][1].time
    
    print(original_outro_time + total_predict_time + total_intro_time)
    predict.tracks[1][1].time += total_intro_time
    outro.tracks[1][1].time = original_outro_time + total_predict_time + total_intro_time
    print(outro.tracks[1][1].time)
    merged_mid = mido.MidiFile()
    merged_mid.ticks_per_beat = intro.ticks_per_beat
    merged_mid.tracks = intro.tracks + predict.tracks + outro.tracks
    merged_mid.save(dissimilar_interpolation + '/merged_predict' + str(i) + '.mid')

In [ ]:
class BeamSearchNode(object):
    def __init__(self, prev_node, wid, logp, length):
        self.prev_node = prev_node
        self.wid = wid
        self.logp = logp
        self.length = length

    def eval(self):
        return self.logp / float(self.length - 1 + 1e-6)
# }}}
import copy
from heapq import heappush, heappop

In [ ]:
def translate_sentence_beam(model, sentence, german, english, device, max_length=1200,beam_width=2,max_dec_steps=25000):
    
    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    eos_token = english.vocab.stoi["<eos>"]
    sos_token = english.vocab.stoi["<sos>"]
    
    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    n_best_list = []
    
     
    #trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

    #first token as input
    trg_tensor = torch.LongTensor(outputs).to(device)
    
    end_nodes = []

    #starting node
    node = BeamSearchNode(prev_node=None, wid=trg_tensor, logp=0, length=1)

    nodes = []

    heappush(nodes, (-node.eval(), id(node), node))
    n_dec_steps = 0

    while True:
        # Give up when decoding takes too long
        if n_dec_steps > max_dec_steps:
            break
        
        # Fetch the best node
        #print([n[2].wid for n in nodes])
        score, _, n = heappop(nodes)
        decoder_input = n.wid
        
        if n.wid.item() == eos_token and n.prev_node is not None:
            end_nodes.append((score, id(n), n))
            # If we reached maximum # of sentences required
            if len(end_nodes) >= beam_width:
                break
            else:
                continue
   
        sequence = [n.wid.item()]
        a = n
        while a.prev_node is not None:
            a = a.prev_node
            sequence.append(a.wid.item())
        sequence = sequence[::-1] # reverse
        
        #print(sequence)
        
        with torch.no_grad():
            output = model(sentence_tensor, torch.LongTensor(sequence).unsqueeze(1).to(device))
        
        # Get top-k from this decoded result
        topk_log_prob, topk_indexes = torch.topk(output, beam_width)
        #print(topk_indexes)
        #print(topk_log_prob)
        # Then, register new top-k nodes
        for new_k in range(beam_width):
            decoded_t = topk_indexes[0][0][new_k].view(1) # (1)
            logp = topk_log_prob[0][0][new_k].item() # float log probability val

            node = BeamSearchNode(prev_node=n,
                                  wid=decoded_t,
                                  logp=n.logp+logp,
                                  length=n.length+1)
            heappush(nodes, (-node.eval(), id(node), node))
        n_dec_steps += beam_width
        #print(n_dec_steps)
    # if there are no end_nodes, retrieve best nodes (they are probably truncated)
    if len(end_nodes) == 0:
        end_nodes = [heappop(nodes) for _ in range(beam_width)]

    # Construct sequences from end_nodes
    n_best_seq_list = []
    for score, _id, n in sorted(end_nodes, key=lambda x: x[0]):
        sequence = [n.wid.item()]
        # back trace from end node
        while n.prev_node is not None:
            n = n.prev_node
            sequence.append(n.wid.item())
        sequence = sequence[::-1] # reverse

        n_best_seq_list.append(sequence)


    # return n_best_seq_list

    translated_sentence = [english.vocab.itos[idx] for idx in n_best_seq_list[0]]

    # remove start token
    return translated_sentence


In [ ]:
def save_vocab(vocab, path):
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

In [ ]:
save_vocab(intro_field.vocab, vocab + '/intro_vocab.pkl')
save_vocab(solo_field.vocab, vocab + '/solo_vocab.pkl')
save_vocab(outro_field.vocab, vocab + '/outro_vocab.pkl')

In [ ]:
def bleu_translate_sentence(model, sentence, german, english, device, max_length=1200):

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    #tokens = [token.lower() for token in sentence.split(' ')]
    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    #tokens.insert(0, german.init_token)
    #tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    #text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(sentence).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence


In [ ]:
from torchtext.data.metrics import bleu_score

def bleu(data, model, german, english, device):
    targets = []
    outputs = []
    print(len(data))
    for example in data:
        #print( vars(example))
        src = vars(example)["intro"]
        trg = vars(example)["solo"]
        
        src = [int(x) for x in src]
        trg = [int(x) for x in trg]
        
        if len(trg) > 1200 or len(src) > 1200:
            continue
        
        prediction = bleu_translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append(trg)
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [ ]:
# running on entire test data takes a while
score = bleu(test[1:10], model, intro_field, solo_field, device)
print(f"Bleu score {score * 100:.2f}")

In [ ]:
# torch.backends.cudnn.enabled = False

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns